In [34]:
import re
import spacy
from spacy_sentiws import spaCySentiWS
import os
nlp = spacy.load('de')
sentiws = spaCySentiWS(sentiws_path=os.path.join(os.getcwd(), 'aux'))
nlp.add_pipe(sentiws)

In [35]:
input_path = os.path.join(os.getcwd(), "input")
with open(os.path.join(input_path, "positive_example5.txt"), "r") as f:
     text = f.read()  
        

In [36]:
def clean_text(text, substitution_dict):
    for key, value in sub_dict.items():
        text = re.sub(key, value, text)
    return text

In [37]:
# Define dictionary with patterns to substitute for cleaning purposes
sub_dict = {
    "\n\n": "\n",
}

In [38]:
text = clean_text(text, sub_dict)

In [39]:
doc = nlp(text)

In [40]:
targets = ["flüchtlinge", "flüchtling", "geflüchtete", "geflüchteter", "geflüchtete", "asylbewerber", "asylsuchender", "asylant", "migrant", "migranten", "migrantin"]

In [41]:
# TODO: It is probably good to give each article an ID in my collection so that I don't have to save the whole doc with the score.
def score_articles(doc):
    results = []
    sentiments = [token._.sentiws for token in doc if token._.sentiws and token.lower_]
    if len(sentiments) > 0:
        score = sum(sentiments) / len(sentiments)
        results.append((doc.text[:100], score))
    return results

In [42]:
def score_sentences(doc, targets):
    """Score sentences that contain any of the target patterns. Returns a list of tuples"""
    compiled_targets = [re.compile(target) for target in targets]
    results = []
    for sentence in doc.sents:
        if any(target.search(sentence.lower_) for target in compiled_targets):
            # I exclude "Flüchtlinge" and synonyms because they are scored as negative sentiment in the model
            sentiments = [token._.sentiws for token in sentence if token._.sentiws and token.lower_ not in targets]
            if len(sentiments) > 0:
                score = sum(sentiments) / len(sentiments)
                results.append((sentence, score))
    return results

In [50]:
def _export_target_sentences(doc, targets, export_path):
    compiled_targets = [re.compile(target) for target in targets]
    with open(export_path, "w", encoding="utf-8") as f:
        for sentence in doc.sents:
            if any(target.search(sentence.lower_) for target in compiled_targets):
                f.write(sentence.text)

In [51]:
score_sentences(doc, targets)

[(Die Nationale Akademie der Wissenschaften fordert zügige Hilfe für traumatisierte Flüchtlinge.,
  0.004),
 (Die altehrwürdige Nationale Akademie der Wissenschaften fordert in einer Stellungnahme schnelle Hilfe für traumatisierte Flüchtlinge.,
  0.060450000000000004),
 (Flüchtlinge hätten häufig immense Gewalt und lebensbedrohliche Situationen erlebt, warnen die Wissenschaftler.,
  -0.5363),
 (Ein Teil der Flüchtlinge ist dadurch nicht in der Lage, den Alltag zu bewältigen, soziale Beziehungen einzugehen oder eine neue Sprache zu erlernen.",
  0.004),
 (Herr Bajbouj, welche Hilfen benötigen traumatisierte Flüchtlinge?, 0.004),
 (Sie fordern gemeinsam mit zwölf weiteren Autoren schnelle Hilfe für traumatisierte Flüchtlinge.,
  0.041633333333333335),
 (Bislang fehlen systematische Untersuchungen, wie viele der Geflüchteten behandlungswürdige Erkrankungen haben.,
  -0.5365),
 (Die könnten anderen Flüchtlingen beibringen, wie man mit Stress umgeht und ihn vermeidet.,
  -0.0048),
 (Psychis

In [52]:
score_articles(doc)

[('"Es müsste viel mehr passieren"\nEin Interview von Félice Gritti\nDie Nationale Akademie der Wissensch',
  -0.10117272727272733)]

In [53]:
export_path = os.path.join(os.getcwd(), "input", "target_sentences.txt")
_export_target_sentences(doc, targets, export_path)

# Citations
- R. Remus, U. Quasthoff & G. Heyer: SentiWS - a Publicly Available German-language Resource for Sentiment Analysis.
In: Proceedings of the 7th International Language Ressources and Evaluation (LREC'10), pp. 1168-1171, 2010